In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [ ]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [ ]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [6]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, # GQN
                                                                   labels=next_states_labels))
    eloss += -tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                     labels=next_states_labels)) # maximize loss
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels)) # minimize loss
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                                   action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1 real
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
                                                                     labels=tf.ones_like(rates))) # max
#     ###################################################### Q(s,a)= r + Q'(s',a')
#     targetQs = rewards + (gamma * aQs2_logits) # DQN/Qlearning
#     eloss += tf.reduce_mean(tf.square(eQs_logits - targetQs)) # real
#     eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # min
#     aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # max
#     eloss += tf.reduce_mean((aQs_logits+aQs2_logits)/2) # min
#     aloss2 += -tf.reduce_mean((aQs_logits+aQs2_logits)/2) # max
#     #################################################### s'', Q' = ~env(s', ~a'): Repeatable 1
#     next_actions_logits = Act(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
#                               reuse=True)
#     next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states_logits, hidden_size=hidden_size, 
#                                    action_size=action_size, state_size=state_size, reuse=True)
#     aQs_logits = aQs2_logits # pervious one
#     aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
# #     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
# #                                                                     labels=tf.zeros_like(rates))) # min
# #     aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
# #                                                                      labels=tf.ones_like(rates))) # max
#     ###################################################### Q(s,a)= r + Q'(s',a')
#     targetQs = rewards + (gamma * aQs2_logits) # DQN/Qlearning
#     eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # min
#     aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # max
#     #################################################### s'', Q' = ~env(s', ~a'): Repeatable 2
#     next_actions_logits = Act(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
#                               reuse=True)
#     next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states_logits, hidden_size=hidden_size, 
#                                    action_size=action_size, state_size=state_size, reuse=True)
#     aQs_logits = aQs2_logits # pervious one
#     aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
# #     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
# #                                                                     labels=tf.zeros_like(rates))) # min
# #     aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
# #                                                                      labels=tf.ones_like(rates))) # max
#     ###################################################### Q(s,a)= r + Q'(s',a')
#     targetQs = rewards + (gamma * aQs2_logits) # DQN/Qlearning
#     eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # min
#     aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # max
    return actions_logits, aloss, eloss, aloss2

In [7]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [8]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [9]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [10]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [11]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [12]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [13]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:22.0000 R:22.0000 rate:0.0440 aloss:0.7073 eloss:1.4588 aloss2:1.4089 exploreP:0.9978
Episode:1 meanR:19.5000 R:17.0000 rate:0.0340 aloss:0.6913 eloss:1.4440 aloss2:1.4117 exploreP:0.9961
Episode:2 meanR:23.3333 R:31.0000 rate:0.0620 aloss:0.7072 eloss:1.4214 aloss2:1.4296 exploreP:0.9931
Episode:3 meanR:21.5000 R:16.0000 rate:0.0320 aloss:0.7015 eloss:1.4272 aloss2:1.4155 exploreP:0.9915
Episode:4 meanR:24.2000 R:35.0000 rate:0.0700 aloss:0.6859 eloss:1.4149 aloss2:1.4173 exploreP:0.9881
Episode:5 meanR:24.6667 R:27.0000 rate:0.0540 aloss:0.6834 eloss:1.3960 aloss2:1.4302 exploreP:0.9855
Episode:6 meanR:22.8571 R:12.0000 rate:0.0240 aloss:0.7074 eloss:1.3928 aloss2:1.4300 exploreP:0.9843
Episode:7 meanR:22.6250 R:21.0000 rate:0.0420 aloss:0.6922 eloss:1.3813 aloss2:1.4316 exploreP:0.9822
Episode:8 meanR:22.6667 R:23.0000 rate:0.0460 aloss:0.7167 eloss:1.3755 aloss2:1.4303 exploreP:0.9800
Episode:9 meanR:22.2000 R:18.0000 rate:0.0360 aloss:0.6970 eloss:1.3828 aloss2:1.4

Episode:80 meanR:21.8519 R:25.0000 rate:0.0500 aloss:0.7176 eloss:0.9858 aloss2:1.7759 exploreP:0.8394
Episode:81 meanR:21.7683 R:15.0000 rate:0.0300 aloss:0.7264 eloss:0.9957 aloss2:1.7670 exploreP:0.8382
Episode:82 meanR:21.7229 R:18.0000 rate:0.0360 aloss:0.7429 eloss:1.0162 aloss2:1.7282 exploreP:0.8367
Episode:83 meanR:21.6429 R:15.0000 rate:0.0300 aloss:0.7379 eloss:0.9865 aloss2:1.7610 exploreP:0.8354
Episode:84 meanR:21.6235 R:20.0000 rate:0.0400 aloss:0.7342 eloss:1.0092 aloss2:1.7623 exploreP:0.8338
Episode:85 meanR:21.6047 R:20.0000 rate:0.0400 aloss:0.7337 eloss:1.0008 aloss2:1.7351 exploreP:0.8321
Episode:86 meanR:21.5862 R:20.0000 rate:0.0400 aloss:0.7552 eloss:0.9968 aloss2:1.7692 exploreP:0.8305
Episode:87 meanR:21.4659 R:11.0000 rate:0.0220 aloss:0.7465 eloss:1.0113 aloss2:1.7389 exploreP:0.8296
Episode:88 meanR:21.5730 R:31.0000 rate:0.0620 aloss:0.7507 eloss:0.9815 aloss2:1.7740 exploreP:0.8271
Episode:89 meanR:21.5667 R:21.0000 rate:0.0420 aloss:0.7392 eloss:0.9977 

Episode:159 meanR:19.6500 R:16.0000 rate:0.0320 aloss:0.7437 eloss:0.6466 aloss2:2.5978 exploreP:0.7188
Episode:160 meanR:19.6600 R:11.0000 rate:0.0220 aloss:0.8179 eloss:0.6676 aloss2:2.4785 exploreP:0.7180
Episode:161 meanR:19.6300 R:15.0000 rate:0.0300 aloss:0.7773 eloss:0.6670 aloss2:2.4567 exploreP:0.7170
Episode:162 meanR:19.7600 R:28.0000 rate:0.0560 aloss:0.7696 eloss:0.6581 aloss2:2.5494 exploreP:0.7150
Episode:163 meanR:19.7900 R:17.0000 rate:0.0340 aloss:0.7544 eloss:0.6491 aloss2:2.5881 exploreP:0.7138
Episode:164 meanR:19.7600 R:13.0000 rate:0.0260 aloss:0.7820 eloss:0.6779 aloss2:2.4905 exploreP:0.7129
Episode:165 meanR:19.7600 R:21.0000 rate:0.0420 aloss:0.7871 eloss:0.6588 aloss2:2.4873 exploreP:0.7114
Episode:166 meanR:19.7500 R:14.0000 rate:0.0280 aloss:0.7607 eloss:0.6401 aloss2:2.6073 exploreP:0.7104
Episode:167 meanR:19.6200 R:13.0000 rate:0.0260 aloss:0.8078 eloss:0.6184 aloss2:2.6410 exploreP:0.7095
Episode:168 meanR:19.6100 R:12.0000 rate:0.0240 aloss:0.7765 elo

Episode:238 meanR:24.1500 R:13.0000 rate:0.0260 aloss:0.6915 eloss:0.5438 aloss2:3.3214 exploreP:0.5938
Episode:239 meanR:24.0400 R:25.0000 rate:0.0500 aloss:0.6789 eloss:0.5113 aloss2:3.4102 exploreP:0.5923
Episode:240 meanR:23.7800 R:11.0000 rate:0.0220 aloss:0.6910 eloss:0.5146 aloss2:3.5531 exploreP:0.5917
Episode:241 meanR:23.5400 R:28.0000 rate:0.0560 aloss:0.6854 eloss:0.5452 aloss2:3.5282 exploreP:0.5900
Episode:242 meanR:23.9400 R:59.0000 rate:0.1180 aloss:0.6884 eloss:0.5214 aloss2:3.4761 exploreP:0.5866
Episode:243 meanR:24.1800 R:35.0000 rate:0.0700 aloss:0.6877 eloss:0.5284 aloss2:3.4479 exploreP:0.5846
Episode:244 meanR:24.5600 R:50.0000 rate:0.1000 aloss:0.6868 eloss:0.4944 aloss2:3.5115 exploreP:0.5818
Episode:245 meanR:24.4600 R:13.0000 rate:0.0260 aloss:0.6991 eloss:0.4600 aloss2:3.7474 exploreP:0.5810
Episode:246 meanR:25.0600 R:69.0000 rate:0.1380 aloss:0.6850 eloss:0.5014 aloss2:3.5237 exploreP:0.5771
Episode:247 meanR:25.5400 R:59.0000 rate:0.1180 aloss:0.6925 elo

Episode:317 meanR:54.2000 R:115.0000 rate:0.2300 aloss:0.6911 eloss:0.5463 aloss2:3.7921 exploreP:0.3760
Episode:318 meanR:54.4400 R:47.0000 rate:0.0940 aloss:0.6825 eloss:0.5195 aloss2:3.7546 exploreP:0.3743
Episode:319 meanR:54.3200 R:44.0000 rate:0.0880 aloss:0.6767 eloss:0.5198 aloss2:3.8035 exploreP:0.3727
Episode:320 meanR:55.3200 R:121.0000 rate:0.2420 aloss:0.6852 eloss:0.4987 aloss2:3.8104 exploreP:0.3683
Episode:321 meanR:54.8400 R:44.0000 rate:0.0880 aloss:0.6719 eloss:0.5172 aloss2:3.7891 exploreP:0.3667
Episode:322 meanR:55.6700 R:129.0000 rate:0.2580 aloss:0.6784 eloss:0.5022 aloss2:3.7978 exploreP:0.3622
Episode:323 meanR:55.7700 R:33.0000 rate:0.0660 aloss:0.6866 eloss:0.4919 aloss2:3.8054 exploreP:0.3610
Episode:324 meanR:56.1600 R:55.0000 rate:0.1100 aloss:0.6806 eloss:0.4916 aloss2:3.8220 exploreP:0.3591
Episode:325 meanR:56.0300 R:52.0000 rate:0.1040 aloss:0.6861 eloss:0.4939 aloss2:3.8203 exploreP:0.3573
Episode:326 meanR:56.1100 R:38.0000 rate:0.0760 aloss:0.6833 

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [21]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
